In [1]:
from sympy import *
init_printing()

In [2]:
from dataclasses import dataclass, field, replace
from functools import cache, cached_property
from typing import Callable, Iterable, Optional, Self

from sympy.series.sequences import RecursiveSeq, sequence


@dataclass(frozen=True)
class Signal:
    recurrence_generator: Callable[[Function, Symbol], Expr]
    initial: Optional[Iterable[Expr]] = None

    @classmethod
    def indefinite(cls) -> Self:
        return cls(lambda y, n: y(n))

    @classmethod
    def constant(cls, expr: Expr) -> Self:
        return cls(lambda y, n: expr)

    @classmethod
    def zero(cls) -> Self:
        return cls.constant(Integer(0))

    @cached_property
    def sequence(self) -> RecursiveSeq:
        y = Function("y")
        n = Symbol("n", integer=True)
        return RecursiveSeq(self.recurrence_generator(y, n), y(n), n, initial=self.initial)

    def __call__(self, i: int) -> Self:
        return Signal(lambda y, n: self.recurrence_generator(y, n+i))

    def __getitem__(self, i: int) -> Expr:
        y = Function("y")
        n = Symbol("n", integer=True)
        return self.recurrence_generator(y, n+i)

    def _expr(self, compute: Callable[[Expr], Expr]) -> Self:
        return Signal(lambda y, n: compute(self.recurrence_generator(y, n)))

    def __add__(self, rhs: Expr) -> Self:
        return self._expr(lambda e: e + rhs)

    def __sub__(self, rhs: Expr) -> Self:
        return self._expr(lambda e: e - rhs)

    def __neg__(self) -> Self:
        return self._expr(lambda e: -e)

    def __mul__(self, rhs: Expr) -> Self:
        return self._expr(lambda e: e * rhs)

    def __div__(self, rhs: Expr) -> Self:
        return self._expr(lambda e: e / rhs)


@dataclass(frozen=True)
class Wave:
    a: Expr
    b: Expr

    def voltage(self) -> Expr:
        return (self.a + self.b) / 2


@dataclass(frozen=True)
class Wdf:
    @cache
    def impedance(self) -> Expr:
        return 0

    @cache
    def admittance(self) -> Expr:
        return 1 / self.impedance()

    def wave(self, a: Signal) -> Wave: ...

    def voltage(self, a: Signal) -> Expr:
        return self.wave(a).voltage()

    def current(self, a: Signal) -> Expr:
        return self.voltage(incident) / self.impedance()

In [3]:
from sympy import Expr


@dataclass(unsafe_hash=True, frozen=True)
class IdealVoltageSource(Wdf):
    Vs: Expr

    @cache
    def wave(self, a: Signal) -> Wave:
        return Wave(a[0], -a[0] + 2 * self.Vs)


@dataclass(unsafe_hash=True, frozen=True)
class ResistiveVoltageSource(Wdf):
    R: Expr
    Vs: Expr

    @cache
    def impedance(self) -> Expr:
        return self.R

    @cache
    def wave(self, a: Signal) -> Wave:
        return Wave(a[0], self.Vs)

In [4]:
from functools import cached_property
from sympy import Expr, Symbol

samplerate = Symbol("F_s", real=True, positive=True)
dt = 1 / samplerate


@dataclass(unsafe_hash=True, frozen=True)
class Resistor(Wdf):
    R: Expr

    @cache
    def impedance(self) -> Expr:
        return self.R

    @cache
    def wave(self, a: Signal) -> Wave:
        return Wave(a[0], Integer(0))


@dataclass(unsafe_hash=True, frozen=True)
class Capacitor(Wdf):
    C: Expr
    fs: Expr = samplerate

    @cache
    def admittance(self) -> Expr:
        return 2 * self.fs * self.C

    @cache
    def impedance(self) -> Expr:
        return 1 / self.admittance()

    @cache
    def wave(self, a: Signal) -> Wave:
        return Wave(a[0], a[-1])

In [5]:
@dataclass(unsafe_hash=True, frozen=True)
class Series(Wdf):
    P1: Wdf
    P2: Wdf

    @cache
    def impedance(self) -> Expr:
        ra = self.P1.impedance()
        rb = self.P2.impedance()
        return ra + rb

    @cache
    def admittance(self) -> Expr:
        return 1 / self.impedance()

    @cache
    def p1_impedance(self) -> Expr:
        return self.P1.impedance() / self.impedance()

    @cache
    def p2_impedance(self) -> Expr:
        return self.P2.impedance() / self.impedance()

    @cache
    def wave(self, a: Signal) -> Wave:
        p1 = self.P1.wave(a(-1))
        p2 = self.P2.wave(a(-1))
        b1 = p1.b - self.p1_impedance() * (a[0] + p1.b + p2.b)

        p1 = self.P1.wave(Signal.constant(b1))
        p2 = self.P2.wave(-a - b1)
        b = p1.b + p2.b
        return Wave(a[0], b)


@dataclass(unsafe_hash=True, frozen=True)
class Parallel(Wdf):
    P1: Wdf
    P2: Wdf

    @cache
    def admittance(self) -> Expr:
        g1 = self.P1.admittance()
        g2 = self.P2.admittance()
        return g1 + g2

    @cache
    def impedance(self) -> Expr:
        return 1 / self.admittance()

    @cache
    def p1_impedance(self) -> Expr:
        return self.P1.admittance() / self.admittance()

    @cache
    def wave(self, a: Signal) -> Wave:
        p1 = self.P1.wave(a(-1))
        p2 = self.P2.wave(a(-1))
        bdiff = p2.b - p1.b
        btemp = -self.p1_impedance() * bdiff
        b = p2.b + btemp
        return Wave(a[0], b)

In [6]:
@dataclass(frozen=True)
class Inverter(Wdf):
    P: Wdf

    @cache
    def impedance(self) -> Expr:
        return self.P.impedance()

    @cache
    def admittance(self) -> Expr:
        return self.P.admittance()

    def wave(self, a: Signal) -> Wave:
        w = self.P.wave(-a)
        return Wave(a[0], -w.b)

In [10]:
r1, r2, vin = symbols("R_1 R_2 V_in", real=True, positive=True)
net =(Series(
    ResistiveVoltageSource(r1, vin),
    Resistor(r2),
))
net.wave(Signal.zero()).voltage()

In [8]:
net = Parallel.from_children(
    ResistiveVoltageSource.named("V_in", "R"),
    Capacitor.named("C")
)

net.incident(0)

AttributeError: type object 'Parallel' has no attribute 'from_children'